[View in Colaboratory](https://colab.research.google.com/github/ruzbro/data-analysis/blob/master/PickupXspaceDistance.ipynb)

This is the query used to build the analytics dataset. I unioned with hubs just in case needed for viz.

```
select --count(m.order_id), count(distinct m.pickup_address_id) , count(distinct m.delivery_address_id )
       'Parcels' as obj_type,
       '' as obj_name, m.date_created_at,
       m.order_id, m.product, m.delivery_courier,  
       m.pickup_address_id, m.delivery_city,
       pu.msg_lat pup_lat, pu.msg_long pup_long,
       m.delivery_address_id, m.pickup_city,
       del.msg_lat del_lat, del.msg_long del_long
  from xa_orders.t_main m
  join kiel.pckup_addr_latlong_3 pu
    on (pu.pickup_address_id = cast(m.pickup_address_id as string) )
  join kiel.del_adr_latlong_3 del
    on (del.delivery_address_id = cast(m.delivery_address_id as string))
 where 1=1
   --and m.pickup_city = m.delivery_city 
   and m.order_status = 'delivered'
   and m.date_created_at >= '2018-06-01'
   and m.date_created_at < '2018-09-01'
union all
select 'Hubs' as obj_type,
       name as obj_name, null,
       null, null, null,  
       null, null,
       latitude pup_lat, longitude pup_long,
       null, null,
       null del_lat, null del_long
  from `mock_geovisualization_dataset.quadx_hub_locations_t`
```



In [0]:
project_id = 'data-insights-176500'

In [0]:
from google.colab import auth
auth.authenticate_user()

In [15]:
%%bigquery --project data-insights-176500 df
SELECT *
FROM `kiel.pickupxspacedistance`

,obj_type,obj_name,date_created_at,order_id,product,delivery_courier,pickup_address_id,delivery_city,pup_lat,pup_long,delivery_address_id,pickup_city,del_lat,del_long
0,Hubs,East Hub,None,NaN,None,None,NaN,None,14.552109,121.098613,NaN,None,NaN,NaN
1,Hubs,Central Hub,None,NaN,None,None,NaN,None,14.536329,121.021195,NaN,None,NaN,NaN
2,Hubs,Northeast Hub,None,NaN,None,None,NaN,None,14.706032,121.071047,NaN,None,NaN,NaN
3,Hubs,Northwest Hub,None,NaN,None,None,NaN,None,14.648021,120.982269,NaN,None,NaN,NaN
4,Hubs,South Hub,None,NaN,None,None,NaN,None,14.451627,120.982990,NaN,None,NaN,NaN
5,Hubs,West Hub,None,NaN,None,None,NaN,None,14.596339,120.975553,NaN,None,NaN,NaN
6,Hubs,Cebu Central Hub,None,NaN,None,None,NaN,None,10.334864,123.941079,NaN,None,NaN,NaN
7,Hubs,CDO Hub,None,NaN,None,None,NaN,None,8.475208,124.673568,NaN,None,NaN,NaN
8,Hubs,Baguio Hub,None,NaN,None,None,NaN,None,16.390409,120.600383,NaN,None,NaN,NaN
9,Hubs,Urdaneta Hub,None,NaN,None,None,NaN,None,15.953914,120.574309,NaN,None,NaN,NaN


In [0]:
!pip install -q shapely

In [0]:
#Set the coordinate of the central hub
from shapely.geometry import Point

lat = 14.5363291
long = 121.0211951

central_hub_coord = Point(long, lat)


In [0]:
#install pyproj for computing distance in km
!pip install -q pyproj

In [24]:
# Testing the distance formula, using pyproj.
# This will compute the great circle distance between two points on the surface of the Earth
import pyproj
geod = pyproj.Geod(ellps='WGS84')

test_lat = 14.552109
test_long = 121.098613

test_point = Point(test_long, test_lat)

angle1,angle2,distance = geod.inv(central_hub_coord.x, central_hub_coord.y, test_point.x, test_point.y)

dist_in_km = distance/1000

dist_in_km

8.524424650081281

In [0]:
# Define a function to do the computations inline for the dataframe
def get_dist_from_center(inlat, inlong):
  geod = pyproj.Geod(ellps='WGS84')

  lat = 14.5363291
  long = 121.0211951

  central_hub_coord = Point(long, lat)
  
  chk_lat = inlat
  chk_long = inlong

  chk_point = Point(chk_long, chk_lat)

  angle1,angle2,distance = geod.inv(central_hub_coord.x, central_hub_coord.y, chk_point.x, chk_point.y)

  dist_in_km = distance/1000

  return(dist_in_km)

In [20]:
# Checking the function
get_dist_from_center(14.5521088, 121.0986135)

8.524472866467482

In [0]:
# Compute the distance for each observations then add to the dataframe

df['dist_to_centralhub'] = df.apply(lambda row: get_dist_from_center(row['pup_lat'], row['pup_long']), axis=1)

In [23]:
# Check what the new df looks like

df

,obj_type,obj_name,date_created_at,order_id,product,delivery_courier,pickup_address_id,delivery_city,pup_lat,pup_long,delivery_address_id,pickup_city,del_lat,del_long,dist_to_centralhub
0,Hubs,East Hub,None,NaN,None,None,NaN,None,14.552109,121.098613,NaN,None,NaN,NaN,8.524473
1,Hubs,Central Hub,None,NaN,None,None,NaN,None,14.536329,121.021195,NaN,None,NaN,NaN,0.000000
2,Hubs,Northeast Hub,None,NaN,None,None,NaN,None,14.706032,121.071047,NaN,None,NaN,NaN,19.529837
3,Hubs,Northwest Hub,None,NaN,None,None,NaN,None,14.648021,120.982269,NaN,None,NaN,NaN,13.050519
4,Hubs,South Hub,None,NaN,None,None,NaN,None,14.451627,120.982990,NaN,None,NaN,NaN,10.236730
5,Hubs,West Hub,None,NaN,None,None,NaN,None,14.596339,120.975553,NaN,None,NaN,NaN,8.263064
6,Hubs,Cebu Central Hub,None,NaN,None,None,NaN,None,10.334864,123.941079,NaN,None,NaN,NaN,562.816385
7,Hubs,CDO Hub,None,NaN,None,None,NaN,None,8.475208,124.673568,NaN,None,NaN,NaN,779.836695
8,Hubs,Baguio Hub,None,NaN,None,None,NaN,None,16.390409,120.600383,NaN,None,NaN,NaN,210.071035
9,Hubs,Urdaneta Hub,None,NaN,None,None,NaN,None,15.953914,120.574309,NaN,None,NaN,NaN,164.039111


In [27]:
# Find out how much distance can produce and average of 500 parcels per day

#This is the total number of days
tot_no_of_days = len(df['date_created_at'].unique().tolist())

83

In [31]:
# Create a new df for observations that have the same pickup and delivery city

df_same_city = df.loc[df['pickup_city'] == df['delivery_city']]


df_same_city.count()

obj_type               15901
obj_name               15901
date_created_at        15901
order_id               15901
product                15901
delivery_courier       15901
pickup_address_id      15901
delivery_city          15901
pup_lat                15901
pup_long               15901
delivery_address_id    15901
pickup_city            15901
del_lat                15901
del_long               15901
dist_to_centralhub     15901
dtype: int64

In [32]:
# Check the new df

df_same_city.head()

,obj_type,obj_name,date_created_at,order_id,product,delivery_courier,pickup_address_id,delivery_city,pup_lat,pup_long,delivery_address_id,pickup_city,del_lat,del_long,dist_to_centralhub
34,Parcels,,2018-06-30,14289741.0,CMO,Quad-X,10000591.0,Pasig City,14.564490,121.088902,13885683.0,Pasig City,14.552620,121.073168,7.934291
37,Parcels,,2018-06-04,13036933.0,CMO,Quad-X,10000591.0,Pasig City,14.564490,121.088902,13885683.0,Pasig City,14.552620,121.073168,7.934291
48,Parcels,,2018-06-01,12901165.0,CMO,Quad-X,10951711.0,Makati City,14.556049,121.008586,25323036.0,Makati City,14.570603,121.022265,2.570416
53,Parcels,,2018-07-12,14774920.0,CMO,Quad-X,10088931.0,Quezon City,14.627499,121.011209,19523690.0,Quezon City,14.614778,121.008564,10.144731
54,Parcels,,2018-06-20,13827668.0,CMO,Quad-X,10088931.0,Quezon City,14.627499,121.011209,19523690.0,Quezon City,14.614778,121.008564,10.144731


In [37]:
# This computes the average parcel per day for orders whose pickup distance from hub is <= n; Where n=10

df_same_city.loc[df_same_city['dist_to_centralhub'] <= 10]['order_id'].count()/83

142.2289156626506

 I'm assuming this will be use for On-demand? 
 
 Shouldn't we compute for this instead? How may parcels are picked up and delivered within ***n*** km of the central hub? 

Or how many orders are picked up and delivered within ***n*** kms of each?

will stop for now here and do the latlong reliability analysis...